## Carga modulo comun

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../common'))

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from purrfect.dataset import EPKADataset, load_partition,save_partition, create_train_valid_loaders
from torch.utils.data import DataLoader

from purrfect.training import train_model,train_validate
import torch.optim as optim

from purrfect.metrics import MetricAccumulator
from purrfect.active_learning import create_new_partition,create_next_partitions, test_model

from sklearn.model_selection import train_test_split
from purrfect.submission import create_submission

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Definición modelo

In [4]:
# Define the double convolution block
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class ChannelAdder(nn.Module):
    def __init__(self):
        super(ChannelAdder, self).__init__()
        # Define Sobel and Laplacian kernels as tensors
        self.sobel_x = torch.tensor([[-1., 0., 1.],
                                     [-2., 0., 2.],
                                     [-1., 0., 1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)
        
        self.sobel_y = torch.tensor([[-1., -2., -1.],
                                     [ 0.,  0.,  0.],
                                     [ 1.,  2.,  1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)
        
        self.laplacian_kernel = torch.tensor([[0.,  1., 0.],
                                              [1., -4., 1.],
                                              [0.,  1., 0.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)

    def forward(self, x):
        epsilon = 1e-8

        # Extract the first, second, and third channels
        first_channel = x[:, 0, :, :].unsqueeze(1)  # e1 (first channel)
        second_channel = x[:, 1, :, :].unsqueeze(1)  # e2 (second channel)
        third_channel = x[:, 2, :, :].unsqueeze(1)  # error (third channel)

        # 1. Compute the first new channel: sqrt(first_channel^2 + second_channel^2)
        new_channel1 = torch.sqrt(first_channel**2 + second_channel**2)

        # 2. Compute the second new channel: 1/2 * arctan(channel2 / channel1)
        new_channel2 = 0.5 * torch.atan(second_channel / (first_channel + epsilon))

        # 3. Compute Sobel gradients and Laplacians for e1 (first_channel)
        grad_e1_x = F.conv2d(first_channel, self.sobel_x, padding=1)
        grad_e1_y = F.conv2d(first_channel, self.sobel_y, padding=1)
        grad_e1_magnitude = torch.sqrt(grad_e1_x**2 + grad_e1_y**2)
        #laplacian_e1 = F.conv2d(first_channel, self.laplacian_kernel, padding=1)

        # 4. Compute Sobel gradients and Laplacians for e2 (second_channel)
        grad_e2_x = F.conv2d(second_channel, self.sobel_x, padding=1)
        grad_e2_y = F.conv2d(second_channel, self.sobel_y, padding=1)
        grad_e2_magnitude = torch.sqrt(grad_e2_x**2 + grad_e2_y**2)
        #laplacian_e2 = F.conv2d(second_channel, self.laplacian_kernel, padding=1)

        # 5. Compute weighted ellipticity channels (e1_weighted, e2_weighted)
        e1_weighted = first_channel / (third_channel + epsilon)
        e2_weighted = second_channel / (third_channel + epsilon)

        # Concatenate all the channels (original and new) into the output tensor
        output = torch.cat([
            x,                 # Original 3 channels
            new_channel1,      # sqrt(channel1^2 + channel2^2)
            new_channel2,      # 1/2 * arctan(channel2 / channel1)
            #grad_e1_x,         # Gradient X of channel1
            #grad_e1_y,         # Gradient Y of channel1
            #grad_e2_x,         # Gradient X of channel2
            #grad_e2_y,         # Gradient Y of channel2
            grad_e1_magnitude, # Gradient magnitude of channel1
            grad_e2_magnitude, # Gradient magnitude of channel2
            #laplacian_e1,      # Laplacian of channel1
            #laplacian_e2,      # Laplacian of channel2
            e1_weighted,       # e1_weighted
            e2_weighted        # e2_weighted
        ], dim=1)

        return output

# Attention Block
class AttentionBlock(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionBlock, self).__init__()
        
        # W_g: for gating signal, i.e., feature map from decoder
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        # W_x: for the skip connection, i.e., feature map from encoder
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        # Psi: attention map
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        # ReLU for non-linearity
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)  # Apply gating to the decoder signal
        x1 = self.W_x(x)  # Apply gating to the encoder signal

        psi = self.relu(g1 + x1)  # Combine and apply non-linearity
        psi = self.psi(psi)  # Generate the attention map

        return x * psi  # Multiply attention map with skip connection

# U-Net Model with Attention
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        # Channel Adder
        self.channel_adder = ChannelAdder()
        
        # Encoder
        self.encoder1 = DoubleConv(in_channels + 6, 64)
        self.encoder2 = DoubleConv(64, 128)
        self.encoder3 = DoubleConv(128, 256)
        self.encoder4 = DoubleConv(256, 512)

        # Bottleneck
        self.bottleneck = DoubleConv(512, 1024)

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.attention4 = AttentionBlock(F_g=512, F_l=512, F_int=256)
        self.decoder4 = DoubleConv(1024, 512)

        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.attention3 = AttentionBlock(F_g=256, F_l=256, F_int=128)
        self.decoder3 = DoubleConv(512, 256)

        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.attention2 = AttentionBlock(F_g=128, F_l=128, F_int=64)
        self.decoder2 = DoubleConv(256, 128)

        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.attention1 = AttentionBlock(F_g=64, F_l=64, F_int=32)
        self.decoder1 = DoubleConv(128, 64)

        # Final output layer
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Add features
        x = self.channel_adder(x)
        
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(F.max_pool2d(e1, 2))
        e3 = self.encoder3(F.max_pool2d(e2, 2))
        e4 = self.encoder4(F.max_pool2d(e3, 2))

        # Bottleneck
        b = self.bottleneck(F.max_pool2d(e4, 2))

        # Decoder with attention and skip connections
        d4 = self.upconv4(b)
        e4 = self.attention4(g=d4, x=e4)  # Apply attention to the skip connection
        d4 = torch.cat((e4, d4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)
        e3 = self.attention3(g=d3, x=e3)  # Apply attention to the skip connection
        d3 = torch.cat((e3, d3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)
        e2 = self.attention2(g=d2, x=e2)  # Apply attention to the skip connection
        d2 = torch.cat((e2, d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        e1 = self.attention1(g=d1, x=e1)  # Apply attention to the skip connection
        d1 = torch.cat((e1, d1), dim=1)
        d1 = self.decoder1(d1)

        # Final output
        out = self.final_conv(d1)
        return out


## Creación particion inicial

In [5]:
#Creacion de particiones train y valid
init_partition = load_partition("partition_1.json")
train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
save_partition("partition_1_train.json","partitions",train_partition)
save_partition("partition_1_val.json","partitions",val_partition)

## Carga modelo

In [6]:
#Define model
model = UNet( 3, 1)
model = model.to(DEVICE)

In [7]:

# Define Loss
criterion = torch.nn.L1Loss()
current_partition = 1

In [8]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  6.19it/s, WMAPE=3.74, DICE=0.128, DPEAKS=102, Loss=0.0123]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:12<00:00,  6.14it/s, WMAPE=2.45, DICE=0.101, DPEAKS=81.3, Loss=0.0079]


Saving best model
Epoch [3/50]


...

Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:12<00:00,  6.12it/s, WMAPE=2.5, DICE=0.11, DPEAKS=112, Loss=0.00821]


Epoch [7/50]


Validate Epoch 7: 100%|██████████| 77/77 [00:12<00:00,  6.20it/s, WMAPE=4.86, DICE=0.182, DPEAKS=142, Loss=0.0179]


early stopping: 3 epochs without improvement
Training complete.


In [9]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: 100%|██████████| 1941/1941 [05:17<00:00,  6.11it/s, WMAPE=2.24, DICE=0.0737, DPEAKS=54.3, Loss=0.00709]


In [10]:
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_2_val: 100%|██████████| 77/77 [00:12<00:00,  6.11it/s, WMAPE=2.27, DICE=0.0737, DPEAKS=56.6, Loss=0.00702]


In [11]:
current_partition=2

In [12]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  6.14it/s, WMAPE=13.2, DICE=0.342, DPEAKS=143, Loss=0.044]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:12<00:00,  6.10it/s, WMAPE=2.71, DICE=0.093, DPEAKS=79.5, Loss=0.00917]


Saving best model
Epoch [3/50]


...

Epoch [5/50]


Validate Epoch 5: 100%|██████████| 77/77 [00:12<00:00,  6.18it/s, WMAPE=2.54, DICE=0.0838, DPEAKS=78.5, Loss=0.00742]


Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:12<00:00,  6.19it/s, WMAPE=2.68, DICE=0.0702, DPEAKS=67.4, Loss=0.00784]


early stopping: 3 epochs without improvement
Training complete.


In [13]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_3_val: 100%|██████████| 77/77 [00:12<00:00,  5.97it/s, WMAPE=2.03, DICE=0.0705, DPEAKS=61.2, Loss=0.00627]


In [14]:
for current_partition in range(3,6):
    best_model_path = os.path.join(
        "models", f"best_model_partition_{current_partition}.pth"
    )
    last_checkpoint_path = os.path.join(
        "models", f"last_checkpoint_partition_{current_partition}.pth"
    )
    train_loader, val_loader = create_train_valid_loaders(
        f"partition_{current_partition}_train.json",
        f"partition_{current_partition}_val.json",
        "partitions",
        batch_size=16,
    )
    optimizer = optim.Adam(model.parameters())
    train_model(
        model,
        train_loader,
        val_loader,
        best_model_path,
        last_checkpoint_path,
        criterion,
        optimizer,
        num_epochs=50,
        device=DEVICE,
        early_stopping_patience=3,
    )
    #Cargar mejor modelo de la particion actual
    model.load_state_dict(torch.load(best_model_path,weights_only=True))
    test_model(model,criterion,device=DEVICE,batch_size=16)
    create_next_partitions(current_partition,model,criterion,device=DEVICE)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  6.24it/s, WMAPE=2.54, DICE=0.0752, DPEAKS=70.8, Loss=0.00749]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:12<00:00,  6.21it/s, WMAPE=2.26, DICE=0.054, DPEAKS=55.4, Loss=0.00687]


Saving best model
Epoch [3/50]


...

Validate Epoch 3: 100%|██████████| 77/77 [00:12<00:00,  6.20it/s, WMAPE=2.53, DICE=0.0435, DPEAKS=36.9, Loss=0.00711]


Epoch [4/50]


Validate Epoch 4: 100%|██████████| 77/77 [00:12<00:00,  6.16it/s, WMAPE=2.8, DICE=0.0969, DPEAKS=90.5, Loss=0.00748]


early stopping: 3 epochs without improvement
Training complete.


Validate Epoch partition_6_val: 100%|██████████| 77/77 [00:13<00:00,  5.83it/s, WMAPE=1.96, DICE=0.0476, DPEAKS=41.1, Loss=0.00563]


In [14]:
current_partition=3
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)

In [15]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:11<00:00,  6.60it/s, WMAPE=2.08, DICE=0.0622, DPEAKS=54.3, Loss=0.00575]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  6.64it/s, WMAPE=5.84, DICE=0.0619, DPEAKS=53, Loss=0.0147]


Epoch [3/50]


...

Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:11<00:00,  6.63it/s, WMAPE=2.08, DICE=0.0511, DPEAKS=46.3, Loss=0.0058]


Epoch [7/50]


Validate Epoch 7: 100%|██████████| 77/77 [00:11<00:00,  6.59it/s, WMAPE=2.2, DICE=0.0616, DPEAKS=55.9, Loss=0.00639]


early stopping: 3 epochs without improvement
Training complete.


In [16]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_4_val: 100%|██████████| 77/77 [00:12<00:00,  6.26it/s, WMAPE=1.87, DICE=0.0494, DPEAKS=32.6, Loss=0.00539]


In [8]:
current_partition=4

In [9]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:11<00:00,  6.47it/s, loss=0.0462]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  7.00it/s, loss=0.0335]


Saving best model
Epoch [3/50]


...

Epoch [16/50]


Validate Epoch 16: 100%|██████████| 77/77 [00:10<00:00,  7.06it/s, loss=0.018] 


Epoch [17/50]


Validate Epoch 17: 100%|██████████| 77/77 [00:10<00:00,  7.05it/s, loss=0.0181]


early stopping: 3 epochs without improvement
Training complete.


In [10]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_5_val: 100%|██████████| 77/77 [00:12<00:00,  6.09it/s, loss=0.0192]


In [7]:
current_partition=5

In [8]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [8/50]


Validate Epoch 8: 100%|██████████| 77/77 [00:11<00:00,  6.61it/s, WMAPE=5.91, DICE=0.289, DPEAKS=146, Loss=0.0178]


Epoch [9/50]


Validate Epoch 9: 100%|██████████| 77/77 [00:11<00:00,  6.56it/s, WMAPE=5.66, DICE=0.289, DPEAKS=150, Loss=0.0196]


early stopping: 3 epochs without improvement
Training complete.


In [9]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_6_val: 100%|██████████| 77/77 [00:12<00:00,  6.01it/s, WMAPE=4.67, DICE=0.272, DPEAKS=128, Loss=0.0176]


In [7]:
current_partition=3
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
model.load_state_dict(torch.load(best_model_path,weights_only=True))
create_submission(model, "partition_3",submission_path="submissions",device=DEVICE)